In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import vpfits

from scipy.ndimage.filters import gaussian_filter
from scipy.signal import argrelextrema

import pymc as mc
from pymc.Matplot import plot
from scipy import stats
from copy import copy
import gc

# Observed Spectrum

Here we test VAMP on a spectrum measured from quasar Q1422+231, at redshift 3.62.

In [ ]:
# cont = pd.read_table('q1422.cont', delim_whitespace=True, header=None)
# cont.columns = ['wavelength', 'velocity', 'flux', 'noise']
# cont.head()

In [ ]:
cont = np.loadtxt('data/q1422.cont')

In [ ]:
vpfit = vpfits.VPfit()
onesigmaerror = 0.02

## Region Detection

We perform region detection on the entire spectrum, dividing it into regions to be fitted.

In [ ]:
min_region_width = 2

regions = vpfits.compute_detection_regions(cont[:,0], cont[:,2], cont[:,3], 
                                          min_region_width=min_region_width)

#regions = vpfit.compute_detection_regions(cont['wavelength'], cont['flux'], cont['noise'], 
#                                          buffer=buffer, min_region_width=min_region_width)

In [ ]:
# region_arrays = []
# region_pixels = []
# for region in regions:
#     start = np.where(cont['wavelength'] == region[0])[0][0]
#     end = np.where(cont['wavelength'] == region[1])[0][0]
#     region_pixels.append([start, end])
#     region_arrays.append([cont['wavelength'][start:end], cont['flux'][start:end]])

In [ ]:
region_arrays = []
region_pixels = []
for region in regions:
    start = np.where(cont[:,0] == region[0])[0][0]
    end = np.where(cont[:,0] == region[1])[0][0]
    region_pixels.append([start, end])
    region_arrays.append([cont[:,0][start:end], cont[:,2][start:end], cont[:,3][start:end]])

In [ ]:
def plot_bracket(x, axis, dir):
    height = .2
    arm_length = 0.2
    axis.plot((x, x), (1-height/2, 1+height/2), color='magenta')

    if dir=='left':
        xarm = x+arm_length
    if dir=='right':
        xarm = x-arm_length

    axis.plot((x, xarm), (1-height/2, 1-height/2), color='magenta')
    axis.plot((x, xarm), (1+height/2, 1+height/2), color='magenta')

In [ ]:
# N = 6

# fig, ax = plt.subplots(N, figsize=(10,10))

# for n in range(N):
    
#     length = len(cont) / N
    
#     lower_lim = n*length
#     upper_lim = n*length+length
    
#     ax[n].plot(cont['wavelength'], cont['flux'], c='black')
    
#     ax[n].set_xlim(cont['wavelength'][lower_lim], cont['wavelength'][upper_lim])

#     for arr in region_arrays:
#         ax[n].plot(arr[0], arr[1], color='blue')

#     for (start, end) in region_pixels:
#         plot_bracket(cont['wavelength'][start], ax[n], 'left')
#         plot_bracket(cont['wavelength'][end], ax[n], 'right')


# plt.show()

In [ ]:
N = 6
fig, ax = plt.subplots(N, figsize=(15,15))

for n in range(N):
    length = len(cont) / N
    
    lower_lim = n*length
    upper_lim = n*length+length
    
    ax[n].plot(cont[:,0], cont[:,2], c='black')
    
    ax[n].set_xlim(cont[:,0][lower_lim], cont[:,0][upper_lim])

    for arr in region_arrays:
        ax[n].plot(arr[0], arr[1], color='blue')

    for (start, end) in region_pixels:
        plot_bracket(cont[:,0][start], ax[n], 'left')
        plot_bracket(cont[:,0][end], ax[n], 'right')

plt.show()

## Fitting

We fit each region separately, determining the optimal number of profiles to fit with.

In [ ]:
iterations = 10000
thin = 15
burn = 1000
for i in range(len(region_arrays)):
    wavelengths = region_arrays[i][0]
    fluxes_orig = region_arrays[i][1]
    fluxes = region_arrays[i][1]
    noise = region_arrays[i][2]
    n = argrelextrema(gaussian_filter(fluxes_orig, 3), np.less)[0].shape[0]
    """Smooth the spectra with a gaussian and find the number of local minima.
    as a safety precaucion, set the initial guess for number of lines to 1 if
    there are less than 4 local minima."""
    if n < 4:
        n = 1
    first = True
    finished = False
    print "Setting initial number of lines to: {}".format(n)
    while not finished:
        bic_array = []
        for j in range(3):
            vpfit_2 = vpfits.VPfit()
            vpfit_2.initialise_model(wavelengths, fluxes, n)
            vpfit_2.map = mc.MAP(vpfit_2.model)
            vpfit_2.mcmc = mc.MCMC(vpfit_2.model)
            vpfit_2.map.fit(iterlim=iterations, tol=1e-3)
            vpfit_2.mcmc.sample(iter=2000, burn=burn, thin=thin, progress_bar=False)
            vpfit_2.map.fit(iterlim=iterations, tol=1e-3)
            vpfit_2.mcmc.sample(iter=2000, burn=burn, thin=thin, progress_bar=False)
            vpfit_2.map.fit(iterlim=iterations, tol=1e-3)
            bic_array.append(vpfit_2.map.BIC)
        if first:
            first = False
            n += 1
            bic_old = vpfit_2.map.BIC
            vpfit_old = copy(vpfit_2)
            del vpfit_2
        else:
            if bic_old > np.average(bic_array):
                print "Old BIC value of {:.2f} is greater than the current {:.2f}.".format(bic_old, np.average(bic_array))
                print "Increasing the number of lines to: {}".format(n+1)
                n += 1
                bic_old = np.average(bic_array)
                vpfit_old = copy(vpfit_2)
                del vpfit_2
            else:
                print "BIC increased with increasing the line number, stopping."
                print "Final n={}.".format(n-1)
                finished = True
    vpfit_old.mcmc.sample(iter=15000, burn=burn, thin=thin, progress_bar=False)
    start = region_pixels[i][0]
    end = region_pixels[i][1]
    vpfit_old.plot(wavelengths, fluxes_orig, n=n-1, start_pix=start, end_pix=end)
    del vpfit_2
    gc.collect()